In [1]:

from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd# import everything ml
import torch
from torch.utils.data import DataLoader, TensorDataset
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
import re
import string
from wordcloud import WordCloud
import nltk
from collections import Counter
import contractions
from tqdm import tqdm
import math
from torch.utils.data import Dataset, DataLoader
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import os
from PIL import Image, ImageFile
from PIL import ImageDraw
import datasets
from datasets import load_dataset
from torch.utils.data import random_split

In [6]:
import os
import shutil

def filter_and_move_images(source_dir, dest_dir, suffix="_i2.png"):
    """
    Finds all files in the source directory ending with a specific suffix
    and moves them to the destination directory.

    Args:
        source_dir (str): The path to the folder containing the original images.
        dest_dir (str): The path to the folder where filtered images will be moved.
        suffix (str): The ending string to filter files by (e.g., "_i2.png").
    """
    # 1. Check if the source directory exists
    if not os.path.isdir(source_dir):
        print(f"❌ Error: Source folder not found at '{source_dir}'")
        return

    # 2. Create the destination directory if it doesn't exist
    if not os.path.exists(dest_dir):
        print(f"Destination folder not found. Creating it at: '{dest_dir}'")
        os.makedirs(dest_dir)

    print(f"🔍 Searching for files ending with '{suffix}' in '{source_dir}'...")

    # 3. Find and move the files
    files_moved = 0
    for filename in os.listdir(source_dir):
        if filename.endswith(suffix):
            source_path = os.path.join(source_dir, filename)
            destination_path = os.path.join(dest_dir, filename)
            
            # Move the file
            shutil.move(source_path, destination_path)
            print(f"  -> Moved '{filename}'")
            files_moved += 1

    print(f"\n✅ Done! Moved a total of {files_moved} files.")


if __name__ == "__main__":
    # --- USER INPUT ---
    # ⬇️⬇️⬇️ **IMPORTANT: UPDATE THESE FOLDER PATHS** ⬇️⬇️⬇️
    
    # The folder where all your original images are currently located.
    source_folder = "./data/terrain_reconstruction/_dataset"
    
    # The folder where you want the filtered '_i2.png' images to go.
    # This will be created if it doesn't exist.
    destination_folder = "./data/terrain_reconstruction/segmentation_maps_folder"

    # ⬆️⬆️⬆️ **IMPORTANT: UPDATE THESE FOLDER PATHS** ⬆️⬆️⬆️

    # filter_and_move_images(source_folder, destination_folder)

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import torchvision.transforms as transforms
from tqdm.auto import tqdm # Import tqdm for progress bars

# ==============================================================================
# 1. Model Architecture (No changes here)
# ==============================================================================

class UNetGenerator(nn.Module):
    def __init__(self, in_channels=3, out_channels=3):
        super(UNetGenerator, self).__init__()
        self.encoder1 = self.conv_block(in_channels, 64, batch_norm=False)
        self.encoder2 = self.conv_block(64, 128)
        self.encoder3 = self.conv_block(128, 256)
        self.encoder4 = self.conv_block(256, 512)
        self.encoder5 = self.conv_block(512, 512)
        self.encoder6 = self.conv_block(512, 512)
        self.encoder7 = self.conv_block(512, 512)
        self.bottleneck = nn.Sequential(nn.Conv2d(512, 512, 4, 2, 1), nn.ReLU())
        self.decoder1 = self.deconv_block(512, 512, dropout=True)
        self.decoder2 = self.deconv_block(1024, 512, dropout=True)
        self.decoder3 = self.deconv_block(1024, 512, dropout=True)
        self.decoder4 = self.deconv_block(1024, 512)
        self.decoder5 = self.deconv_block(1024, 256)
        self.decoder6 = self.deconv_block(512, 128)
        self.decoder7 = self.deconv_block(256, 64)
        self.final_layer = nn.Sequential(nn.ConvTranspose2d(128, out_channels, 4, 2, 1), nn.Tanh())
    def conv_block(self, in_channels, out_channels, batch_norm=True):
        layers = [nn.Conv2d(in_channels, out_channels, 4, 2, 1, bias=False), nn.LeakyReLU(0.2)]
        if batch_norm: layers.insert(1, nn.BatchNorm2d(out_channels))
        return nn.Sequential(*layers)
    def deconv_block(self, in_channels, out_channels, dropout=False):
        layers = [nn.ConvTranspose2d(in_channels, out_channels, 4, 2, 1, bias=False), nn.BatchNorm2d(out_channels), nn.ReLU()]
        if dropout: layers.append(nn.Dropout(0.5))
        return nn.Sequential(*layers)
    def forward(self, x):
        e1=self.encoder1(x);e2=self.encoder2(e1);e3=self.encoder3(e2);e4=self.encoder4(e3)
        e5=self.encoder5(e4);e6=self.encoder6(e5);e7=self.encoder7(e6)
        bottle=self.bottleneck(e7)
        d1=torch.cat([self.decoder1(bottle),e7],1);d2=torch.cat([self.decoder2(d1),e6],1)
        d3=torch.cat([self.decoder3(d2),e5],1);d4=torch.cat([self.decoder4(d3),e4],1)
        d5=torch.cat([self.decoder5(d4),e3],1);d6=torch.cat([self.decoder6(d5),e2],1)
        d7=torch.cat([self.decoder7(d6),e1],1)
        return self.final_layer(d7)

class PatchDiscriminator(nn.Module):
    def __init__(self, in_channels=6):
        super(PatchDiscriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(in_channels, 64, 4, 2, 1), nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, 4, 2, 1, bias=False), nn.BatchNorm2d(128), nn.LeakyReLU(0.2),
            nn.Conv2d(128, 256, 4, 2, 1, bias=False), nn.BatchNorm2d(256), nn.LeakyReLU(0.2),
            nn.Conv2d(256, 512, 4, 1, 1, bias=False), nn.BatchNorm2d(512), nn.LeakyReLU(0.2),
            nn.Conv2d(512, 1, 4, 1, 1)
        )
    def forward(self, x, y=None):
        out = torch.cat([x, y], dim=1) if y is not None else x
        return self.model(out)

# ==============================================================================
# 2. Custom Dataset for Your Images
# ==============================================================================

class CustomImageDataset(Dataset):
    def __init__(self, image_dir, image_size=256):
        self.image_dir = image_dir
        self.image_paths = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
        self.transform = transforms.Compose([
            transforms.Resize((image_size, image_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
        ])
    def __len__(self):
        return len(self.image_paths)
    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('RGB')
        return self.transform(image)

# ==============================================================================
# 3. Part 1: Seamless Tile Generator
# ==============================================================================

# --- Hyperparameters & User Input ---
image_dir = "../../data/terrain_reconstruction/segmentation_maps_folder"
image_size = 256
noise_dim = 100
batch_size_seamless = 16
epochs_seamless = 100
lr_seamless = 0.0002
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("mps")

if not os.path.exists(image_dir) or not os.listdir(image_dir):
    print(f"❌ Error: Directory not found or is empty at '{image_dir}'.")
    print("Please create the folder, add your images, or update the 'image_dir' variable.")
else:
    seamless_dataset = CustomImageDataset(image_dir=image_dir, image_size=image_size)
    seamless_loader = DataLoader(seamless_dataset, batch_size=batch_size_seamless, shuffle=True)

    seamless_gen = UNetGenerator(in_channels=noise_dim, out_channels=3).to(device)
    seamless_disc = PatchDiscriminator(in_channels=3).to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer_gen_seamless = optim.Adam(seamless_gen.parameters(), lr=lr_seamless, betas=(0.5, 0.999))
    optimizer_disc_seamless = optim.Adam(seamless_disc.parameters(), lr=lr_seamless, betas=(0.5, 0.999))

    print(f"🚀 Starting Seamless GAN Training with {len(seamless_dataset)} images...")
    for epoch in range(epochs_seamless):
        # --- ✨ ADDED: tqdm progress bar ---
        progress_bar = tqdm(enumerate(seamless_loader), total=len(seamless_loader), desc=f"Epoch {epoch+1}/{epochs_seamless}")
        for i, real_images in progress_bar:
            real_images = real_images.to(device)
            noise = torch.randn(real_images.size(0), noise_dim, 1, 1, device=device).repeat(1, 1, image_size, image_size)
            
            optimizer_disc_seamless.zero_grad()
            fake_images = seamless_gen(noise)
            disc_real = seamless_disc(real_images)
            disc_fake = seamless_disc(fake_images.detach())
            loss_disc = (criterion(disc_real, torch.ones_like(disc_real)) + criterion(disc_fake, torch.zeros_like(disc_fake)))/2
            loss_disc.backward()
            optimizer_disc_seamless.step()

            optimizer_gen_seamless.zero_grad()
            disc_fake = seamless_disc(fake_images)
            loss_gen = criterion(disc_fake, torch.ones_like(disc_fake))
            loss_gen.backward()
            optimizer_gen_seamless.step()
            
            # --- ✨ ADDED: Update progress bar with current loss ---
            progress_bar.set_postfix(loss_D=f"{loss_disc.item():.4f}", loss_G=f"{loss_gen.item():.4f}")

# ==============================================================================
# 4. Part 2: Conditional Stitching GAN
# ==============================================================================

class StitchingDataset(Dataset):
    def __init__(self, image_paths, size, stitch_width):
        self.image_paths = image_paths
        self.size = size
        self.stitch_width = stitch_width
        self.transform = transforms.Compose([
            transforms.Resize((size, size)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
        ])
    def __len__(self):
        return len(self.image_paths)
    def __getitem__(self, idx):
        full_image = Image.open(self.image_paths[idx]).convert('RGB')
        full_image = self.transform(full_image)
        conditional_strip = torch.zeros_like(full_image)
        conditional_strip[:, :, :self.stitch_width] = full_image[:, :, :self.stitch_width]
        return conditional_strip, full_image

stitch_width = 32
batch_size_stitch = 16
epochs_stitch = 100
lr_stitch = 0.0002

if os.path.exists(image_dir) and os.listdir(image_dir):
    stitch_dataset = StitchingDataset(seamless_dataset.image_paths, image_size, stitch_width)
    stitch_loader = DataLoader(stitch_dataset, batch_size=batch_size_stitch, shuffle=True)

    stitch_gen = UNetGenerator(in_channels=3, out_channels=3).to(device)
    stitch_disc = PatchDiscriminator(in_channels=6).to(device)
    optimizer_gen_stitch = optim.Adam(stitch_gen.parameters(), lr=lr_stitch, betas=(0.5, 0.999))
    optimizer_disc_stitch = optim.Adam(stitch_disc.parameters(), lr=lr_stitch, betas=(0.5, 0.999))
    l1_loss = nn.L1Loss()

    print(f"\n🚀 Starting Stitching GAN Training with {len(stitch_dataset)} images...")
    for epoch in range(epochs_stitch):
        # --- ✨ ADDED: tqdm progress bar ---
        progress_bar = tqdm(enumerate(stitch_loader), total=len(stitch_loader), desc=f"Epoch {epoch+1}/{epochs_stitch}")
        for i, (cond_image, real_image) in progress_bar:
            cond_image, real_image = cond_image.to(device), real_image.to(device)

            optimizer_disc_stitch.zero_grad()
            fake_image = stitch_gen(cond_image)
            disc_real = stitch_disc(cond_image, real_image)
            disc_fake = stitch_disc(cond_image, fake_image.detach())
            loss_disc = (criterion(disc_real, torch.ones_like(disc_real)) + criterion(disc_fake, torch.zeros_like(disc_fake))) / 2
            loss_disc.backward()
            optimizer_disc_stitch.step()

            optimizer_gen_stitch.zero_grad()
            disc_fake = stitch_disc(cond_image, fake_image)
            loss_gen_gan = criterion(disc_fake, torch.ones_like(disc_fake))
            loss_gen_l1 = l1_loss(fake_image, real_image) * 100
            loss_gen = loss_gen_gan + loss_gen_l1
            loss_gen.backward()
            optimizer_gen_stitch.step()
            
            # --- ✨ ADDED: Update progress bar with current loss ---
            progress_bar.set_postfix(loss_D=f"{loss_disc.item():.4f}", loss_G=f"{loss_gen.item():.4f}", loss_L1=f"{loss_gen_l1.item():.4f}")

    print("\n✅ Stitching GAN Training Complete!")

❌ Error: Directory not found or is empty at '../../data/terrain_reconstruction/segmentation_maps_folder'.
Please create the folder, add your images, or update the 'image_dir' variable.


In [ ]:


torch.save(stitch_gen.state_dict(), "../../models/stitch_gen.pth")

In [4]:
with torch.no_grad():
    noise = torch.randn(1, noise_dim, 1, 1, device=device).repeat(1, 1, image_size, image_size)
    left_tile = seamless_gen(noise).detach().cpu()
    right_cond = torch.zeros_like(left_tile)
    right_cond[:, :, :stitch_width] = left_tile[:, :, -stitch_width:]
    right_tile = stitch_gen(right_cond.to(device)).detach().cpu()

    # Debugging: Print shapes
    print("Left tile shape:", left_tile.shape)
    print("Right tile shape:", right_tile.shape)

    # Ensure dimensions match for concatenation
    left_tile_cropped = left_tile[:, :, :, :-stitch_width]
    print("Left tile cropped shape:", left_tile_cropped.shape)

    stitched_image = torch.cat([left_tile_cropped, right_tile], dim=3)

    plt.figure(figsize=(12, 6))
    stitched_image_display = (stitched_image[0].permute(1, 2, 0) + 1) / 2
    plt.imshow(stitched_image_display.clamp(0, 1))
    plt.title('Two Generated Maps Stitched Together')
    plt.axvline(x=image_size - stitch_width - 0.5, color='r', linestyle='--')
    plt.show()

NameError: name 'stitch_width' is not defined

In [7]:
import os
import tensorflow as tf
import numpy as np

# Path to your prepared segmentation maps
DATA_FOLDER = "./data/terrain_reconstruction/segmentation_maps_folder"

# Get a list of all your image file paths once at the start
image_paths = [os.path.join(DATA_FOLDER, f) for f in os.listdir(DATA_FOLDER) if f.endswith('.png')]
print(f"Found {len(image_paths)} images for training.")

def load_real_samples(batch_size):
    """
    Loads a random batch of real terrain images from your folder.
    """
    # Choose random indices to create a batch
    random_indices = np.random.randint(0, len(image_paths), batch_size)
    
    batch_images = []
    for i in random_indices:
        # Load and decode the image
        img = tf.io.read_file(image_paths[i])
        # Decode as grayscale, channels=1
        img = tf.image.decode_png(img, channels=1) 
        # Resize to the model's expected input
        img = tf.image.resize(img, [IMG_ROWS, IMG_COLS]) 
        batch_images.append(img)
        
    # Convert list of tensors to a single tensor
    X = tf.stack(batch_images)
    
    # Normalize pixel values from [0, 255] to [-1, 1]
    X = (X - 127.5) / 127.5
    
    # Create "real" class labels (value of 1)
    y = np.ones((batch_size, 1))
    
    return X, y

# --- In your training loop ---
# Replace the old call:
# X_real, y_real = generate_real_samples(half_batch)

# With the new one:
# X_real, y_real = load_real_samples(half_batch)

2025-07-31 15:03:40.013933: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-31 15:03:40.116910: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753999420.158971 2567133 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753999420.170998 2567133 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753999420.259791 2567133 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Found 5000 images for training.


In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

## 1. Define Model & Data Parameters

# Image dimensions
IMG_ROWS = 256
IMG_COLS = 256
CHANNELS = 1 # Grayscale segmentation maps

# Latent space dimension (input for the generator)
LATENT_DIM = 100

# Path to your prepared segmentation maps
DATA_FOLDER = "./data/terrain_reconstruction/segmentation_maps_folder"

# Get a list of all your image file paths
try:
    image_paths = [os.path.join(DATA_FOLDER, f) for f in os.listdir(DATA_FOLDER) if f.endswith('.png')]
    print(f"✅ Found {len(image_paths)} images for training in '{DATA_FOLDER}'.")
    assert len(image_paths) > 0, "No images found. Did you run the data preparation script?"
except (FileNotFoundError, AssertionError) as e:
    print(f"❌ Error: {e}")
    print("Please ensure the data preparation script has been run and the path is correct.")
    # Exit or handle error appropriately in a real script
    image_paths = []


## 2. Build the Generator Model

def build_generator(latent_dim):
    """Builds the Generator model."""
    model = Sequential(name='Generator')
    n_nodes = 256 * 16 * 16
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((16, 16, 256)))
    # Upsample to 32x32, 64x64, 128x128, 256x256
    for i in range(4):
        model.add(Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))
    # Output layer
    model.add(Conv2D(CHANNELS, (7, 7), activation='tanh', padding='same'))
    return model

## 3. Build the Discriminator Model

def build_discriminator(img_shape):
    """Builds the Discriminator model."""
    model = Sequential(name='Discriminator')
    model.add(Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=img_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.3))
    model.add(Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.3))
    model.add(Conv2D(256, (5, 5), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.3))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

## 4. Build the Combined GAN Model

def build_gan(generator, discriminator):
    """Builds the combined GAN model."""
    discriminator.trainable = False
    model = Sequential(name='GAN')
    model.add(generator)
    model.add(discriminator)
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

## 5. Define Data Loading and Helper Functions

def load_real_samples(batch_size):
    """Loads a random batch of real terrain images from your folder."""
    random_indices = np.random.randint(0, len(image_paths), batch_size)
    batch_images = []
    for i in random_indices:
        img = tf.io.read_file(image_paths[i])
        img = tf.image.decode_png(img, channels=CHANNELS)
        img = tf.image.resize(img, [IMG_ROWS, IMG_COLS])
        batch_images.append(img)
    X = tf.stack(batch_images)
    X = (X - 127.5) / 127.5 # Normalize to [-1, 1]
    y = np.ones((batch_size, 1))
    return X, y

def generate_latent_points(latent_dim, n_samples):
    """Generates points in the latent space."""
    return np.random.randn(latent_dim * n_samples).reshape(n_samples, latent_dim)

def generate_fake_samples(generator, latent_dim, n_samples):
    """Generates 'fake' samples using the generator."""
    x_input = generate_latent_points(latent_dim, n_samples)
    X = generator.predict(x_input, verbose=0)
    y = np.zeros((n_samples, 1))
    return X, y

def save_plot(examples, epoch, n=5):
    """Saves a plot of generated images."""
    examples = (examples + 1) / 2.0 # Scale from [-1, 1] to [0, 1]
    plt.figure(figsize=(10, 10))
    for i in range(n * n):
        plt.subplot(n, n, 1 + i)
        plt.axis('off')
        plt.imshow(examples[i, :, :, 0], cmap='terrain')
    filename = f'generated_plot_e{epoch+1:03d}.png'
    plt.savefig(filename)
    plt.close()

## 6. Define the Main Training Loop

def train(g_model, d_model, gan_model, latent_dim, n_epochs=200, n_batch=32):
    """Main training loop for the GAN."""
    bat_per_epo = int(len(image_paths) / n_batch)
    half_batch = int(n_batch / 2)
    
    print("\n--- Starting Training ---")
    for i in range(n_epochs):
        for j in range(bat_per_epo):
            # Train discriminator
            X_real, y_real = load_real_samples(half_batch)
            d_loss1, _ = d_model.train_on_batch(X_real, y_real)
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)

            # Train generator
            X_gan = generate_latent_points(latent_dim, n_batch)
            y_gan = np.ones((n_batch, 1))
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            
            # Summarize loss
            if (j + 1) % (bat_per_epo // 2) == 0: # Print twice per epoch
                 print(f'> Epoch {i+1}/{n_epochs}, Batch {j+1}/{bat_per_epo}, d_loss=[{d_loss1:.3f},{d_loss2:.3f}], g_loss={g_loss:.3f}')

        # Evaluate and save progress at the end of each epoch
        if (i + 1) % 10 == 0: # Every 10 epochs
            print(f"\n--- Saving progress for epoch {i+1} ---")
            X_real_eval, y_real_eval = load_real_samples(n_batch)
            _, acc_real = d_model.evaluate(X_real_eval, y_real_eval, verbose=0)
            X_fake_eval, y_fake_eval = generate_fake_samples(g_model, latent_dim, n_batch)
            _, acc_fake = d_model.evaluate(X_fake_eval, y_fake_eval, verbose=0)
            print(f'> Accuracy real: {acc_real*100:.2f}%, fake: {acc_fake*100:.2f}%')
            save_plot(X_fake_eval, i, n=4)
            g_model.save(f'generator_model_{i+1:03d}.h5')

## 7. Create Models and Start Training

# Create the discriminator
d_model = build_discriminator((IMG_ROWS, IMG_COLS, CHANNELS))
# Create the generator
g_model = build_generator(LATENT_DIM)
# Create the GAN
gan_model = build_gan(g_model, d_model)

# Start training
# Note: n_epochs=200 is a good start. Real-world results might need 1000s of epochs.
if image_paths:
    train(g_model, d_model, gan_model, LATENT_DIM, n_epochs=20, n_batch=32)
else:
    print("\nSkipping training because no image data was loaded.")

2025-07-31 15:07:16.242312: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-31 15:07:16.250291: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753999636.258731 2579296 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753999636.261348 2579296 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753999636.269156 2579296 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

✅ Found 5000 images for training in './data/terrain_reconstruction/segmentation_maps_folder'.


/home/software/Documents/.rianstuff/cosmos/env/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1753999637.346182 2579296 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5069 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060, pci bus id: 0000:01:00.0, compute capability: 8.9
/home/software/Documents/.rianstuff/cosmos/env/lib/python3.11/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
/home/software/Documents/.rianstuff/cosmos/env/lib/python3.11/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argu


--- Starting Training ---


I0000 00:00:1753999637.934435 2579296 service.cc:152] XLA service 0x5cb3fe9938a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753999637.934447 2579296 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 4060, Compute Capability 8.9
2025-07-31 15:07:17.945284: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1753999637.964322 2579296 cuda_dnn.cc:529] Loaded cuDNN version 90501
I0000 00:00:1753999639.348248 2579296 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2025-07-31 15:07:21.546339: E external/local_xla/xla/service/slow_operation_alarm.cc:73] Trying algorithm eng0{} for conv %cudnn-conv-bw-input.6 = (f32[16,128,128,128]{3,2,1,0}, u8[0]{0}) custom-call(f32[16,128,64,64]{3,2,1,0} %bitcast.581, f32[128,128,4,4]{3,2,1,0} %bitcast.5

> Epoch 1/20, Batch 78/156, d_loss=[0.687,0.687], g_loss=0.258
> Epoch 1/20, Batch 156/156, d_loss=[0.692,0.692], g_loss=0.153
> Epoch 2/20, Batch 78/156, d_loss=[0.707,0.707], g_loss=0.108
> Epoch 2/20, Batch 156/156, d_loss=[0.749,0.750], g_loss=0.084
> Epoch 3/20, Batch 78/156, d_loss=[0.856,0.858], g_loss=0.069
> Epoch 3/20, Batch 156/156, d_loss=[1.044,1.048], g_loss=0.058
> Epoch 4/20, Batch 78/156, d_loss=[1.269,1.273], g_loss=0.050
> Epoch 4/20, Batch 156/156, d_loss=[1.485,1.489], g_loss=0.044
> Epoch 5/20, Batch 78/156, d_loss=[1.677,1.680], g_loss=0.040
> Epoch 5/20, Batch 156/156, d_loss=[1.843,1.846], g_loss=0.036
> Epoch 6/20, Batch 78/156, d_loss=[1.987,1.990], g_loss=0.033
> Epoch 6/20, Batch 156/156, d_loss=[2.114,2.116], g_loss=0.030
> Epoch 7/20, Batch 78/156, d_loss=[2.225,2.228], g_loss=0.028
> Epoch 7/20, Batch 156/156, d_loss=[2.325,2.327], g_loss=0.026
> Epoch 8/20, Batch 78/156, d_loss=[2.415,2.417], g_loss=0.025
> Epoch 8/20, Batch 156/156, d_loss=[2.497,2.499

> Accuracy real: 100.00%, fake: 0.00%
> Epoch 11/20, Batch 78/156, d_loss=[3.951,3.972], g_loss=0.018
> Epoch 11/20, Batch 156/156, d_loss=[3.954,3.965], g_loss=0.017
> Epoch 12/20, Batch 78/156, d_loss=[3.966,3.974], g_loss=0.016
> Epoch 12/20, Batch 156/156, d_loss=[3.980,3.985], g_loss=0.016
> Epoch 13/20, Batch 78/156, d_loss=[3.994,3.999], g_loss=0.015
> Epoch 13/20, Batch 156/156, d_loss=[4.009,4.012], g_loss=0.015
> Epoch 14/20, Batch 78/156, d_loss=[4.023,4.026], g_loss=0.014
> Epoch 14/20, Batch 156/156, d_loss=[4.037,4.039], g_loss=0.014
> Epoch 15/20, Batch 78/156, d_loss=[4.050,4.053], g_loss=0.013
> Epoch 15/20, Batch 156/156, d_loss=[4.063,4.065], g_loss=0.013
> Epoch 16/20, Batch 78/156, d_loss=[4.076,4.078], g_loss=0.012
> Epoch 16/20, Batch 156/156, d_loss=[4.089,4.091], g_loss=0.012
> Epoch 17/20, Batch 78/156, d_loss=[4.101,4.103], g_loss=0.012
> Epoch 17/20, Batch 156/156, d_loss=[4.113,4.115], g_loss=0.011
> Epoch 18/20, Batch 78/156, d_loss=[4.125,4.127], g_loss=0

> Accuracy real: 93.75%, fake: 0.00%
